In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import glob
import os
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from scipy.spatial import distance 
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Load Each Predicted car position sheet (16 Features) fill nans with 0 Pad to 100

In [ ]:
start_folder = 'CSVsPred'
csvs_attackers = glob.glob('{}/*_attacker.csv'.format(start_folder))
df_attackers = []

for c in csvs_attackers:
    df = pd.read_csv(c).drop(['Unnamed: 0','Unnamed: 0.1', 'time', 'sender', 'attackerType'], axis=1).fillna(0)

    df = np.asarray(df).astype('float32')
    df = np.pad(df, ((0,100-df.shape[0]),(0,0)), 'constant')
    if df.shape[1] == 16:

        df_attackers.append(df)
    
print(len(df_attackers))
df_attackers = np.asarray(df_attackers)
np.save('CSVsPred/np_attackers_pred.npy', df_attackers)


In [3]:
start_folder = 'CSVsPred'
csvs_normal = glob.glob('{}/*_normal.csv'.format(start_folder))
df_normal = []

for c in csvs_normal:
    df = pd.read_csv(c).drop(['Unnamed: 0','Unnamed: 0.1', 'time', 'sender', 'attackerType'], axis=1).fillna(0)

    df = np.asarray(df).astype('float32')
    df = np.pad(df, ((0,100-df.shape[0]),(0,0)), 'constant')
    if df.shape[1] == 16:
        df_normal.append(df)
    
print(len(df_normal))
# df_normal = np.asarray(df_normal)
df_normal = np.asarray(df_normal)
np.save('CSVsPred/np_normal_pred.npy', df_normal)

14312


In [ ]:
df_normal[2]

# Loads each whole scenario as a sheet

In [ ]:
start_folder = 'AllCSVs'
csvs_scen = glob.glob(r'{}\*.csv'.format(start_folder))
df_scen = []

for c in csvs_scen:
    df = pd.read_csv(c).drop(['type','messageID','spd_noise0','spd_noise1','spd_noise2' ,'pos_noise0' ,'pos_noise1' ,'pos_noise2', 'spd2', 'pos2'], axis=1)
    df_scen.append(df)
print(len(df_scen))
df_scen[0][0:100]

In [ ]:
for df in df_scen:
    scaler = StandardScaler()
    df[['pos0', 'pos1']] = scaler.fit_transform(df[['pos0', 'pos1']])
    df[['spd0', 'spd1']] = scaler.fit_transform(df[['spd0', 'spd1']])
df_scen[2][0:100]
    

In [ ]:
# Define a function to find the two closest rows above a given row based on A and B values
def find_closest_rows_above(df, row):
    # select a row to find closest rows to
    given_row = row

    # Find the index of the given row
    row_index = df.index.get_loc(given_row.name)

    # Slice the DataFrame to only consider rows above the given row
    above_df = df.iloc[max(0, row_index-2000):row_index]

    if len(above_df) == 0:
        return pd.DataFrame()

    # select the A and B columns of the DataFrame
    df_ab = above_df[['time', 'pos0', 'pos1']]

    # calculate the Euclidean distance between the given row and all other rows based on A and B values
    distances = distance.cdist(df_ab, [given_row[['time', 'pos0', 'pos1']]], metric='euclidean')

    # get the indices of rows sorted by distance
    sorted_indices = distances.argsort(axis=0).flatten()

    # get the closest rows as a DataFrame
    closest_rows = df.iloc[sorted_indices]
    
    # Create a Boolean mask to select the rows that contain the value. Ensures that senders are not the same
    mask = closest_rows['sender'] == search_row.sender

    # Drop the rows that match the mask
    closest_rows.drop(closest_rows[mask].index, inplace=True)

    return closest_rows

In [ ]:
df_closest = []
p = 0
for df in df_scen:
    df_copy = df.copy()
    for i in range(2, len(df_copy)): #len(df)
#         print(i)
#         display(search_row)
        search_row = df_copy.iloc[i] 
        closest_rows = find_closest_rows_above(df.copy(), search_row).reset_index()
        closest_car_nums = [0,1]
        feats = ['pos0','pos1','spd0','spd1']
#         display(closest_rows)
        for num in closest_car_nums:
            for f in feats:
                df_copy.loc[i, f'{f}Close{num}'] = closest_rows.loc[num, f]
    
    df_copy.to_csv(r'C:\Users\aidan\Desktop\5G-BSM-Threat-Detection\ScenarioScaled\scenario_{}.csv'.format(p))  
    df_closest.append(df_copy)
    p+=1

# ----------------------------------------Make Numpy-----------------------------------------------

In [ ]:
# Next Step **Chop out the first 2 rows of the df**
start_folder = 'ScenarioScaled'#'ClosestCarCSVsModel2'
csvs = glob.glob(r'{}\*.csv'.format(start_folder))
df_list = []

for c in csvs:
    df = pd.read_csv(c)
    for i in np.unique(df.sender):
        df_list.append(df[df.sender.isin([i])].copy())
    #     df[df.ID.isin([subs[0]])&df.Vigil.isin(vigils[0])].copy()
print(len(df_list))
# df_list[0]

# BSM Length 10 (Windowed, Slide 9)

In [ ]:
df_attacker = []
df_normal = []
window_size = 20
overlap_size = 19
df = df_list[0:10]
for j in df:
#     print(j)
    if j.attackerType.any() == 0:
        j = j.iloc[:,4:].fillna(0)#4
        for start_idx in range(0, len(j)-window_size, window_size - overlap_size):
            end_idx = start_idx + window_size
            df_temp = j.iloc[start_idx:end_idx]
#             display(df_chunk)
            scaler = StandardScaler()
            df_temp[['pos0', 'pos1']] = scaler.fit_transform(df_temp[['pos0', 'pos1']])
            df_temp[['spd0', 'spd1']] = scaler.fit_transform(df_temp[['spd0', 'spd1']])
            df_temp[['pos0Close0', 'pos1Close0']] = scaler.fit_transform(df_temp[['pos0Close0', 'pos1Close0']])
            df_temp[['spd0Close0', 'spd1Close0']] = scaler.fit_transform(df_temp[['spd0Close0', 'spd1Close0']])
            df_temp[['pos0Close1', 'pos1Close1']] = scaler.fit_transform(df_temp[['pos0Close1', 'pos1Close1']])
            df_temp[['spd0Close1', 'spd1Close1']] = scaler.fit_transform(df_temp[['spd0Close1', 'spd1Close1']])
            
            df_normal.append(df_temp)
        
    
    elif j.attackerType.any() == 1:
        j = j.iloc[:,4:].fillna(0)
        for start_idx in range(0, len(j)-window_size, window_size - overlap_size):
            end_idx = start_idx + window_size
            df_temp = j.iloc[start_idx:end_idx]
#             display(df_chunk)
            scaler = StandardScaler()
            df_temp[['pos0', 'pos1']] = scaler.fit_transform(df_temp[['pos0', 'pos1']])
            df_temp[['spd0', 'spd1']] = scaler.fit_transform(df_temp[['spd0', 'spd1']])
            df_temp[['pos0Close0', 'pos1Close0']] = scaler.fit_transform(df_temp[['pos0Close0', 'pos1Close0']])
            df_temp[['spd0Close0', 'spd1Close0']] = scaler.fit_transform(df_temp[['spd0Close0', 'spd1Close0']])
            df_temp[['pos0Close1', 'pos1Close1']] = scaler.fit_transform(df_temp[['pos0Close1', 'pos1Close1']])
            df_temp[['spd0Close1', 'spd1Close1']] = scaler.fit_transform(df_temp[['spd0Close1', 'spd1Close1']])
            
            df_attacker.append(df_temp)
        

# df_normal = np.asarray(df_normal)
# df_attacker = np.asarray(df_attacker)
display(df_normal[0])

In [ ]:
flat = np.asarray(df_normal[0])

flat.flatten().shape

In [ ]:
df_normal.shape, df_attacker.shape

In [ ]:
np.save(r'C:\Users\aidan\Desktop\5G-BSM-Threat-Detection\ClosestCarCSVsModel2\np_normal_len50.npy', df_normal)
np.save(r'C:\Users\aidan\Desktop\5G-BSM-Threat-Detection\ClosestCarCSVsModel2\np_attacker_len50.npy', df_attacker)


# BSM Length 100 (Full)

In [ ]:
df_list[0].iloc[:,4:].fillna(0)
df_temp = np.asarray(df_temp).astype('float32')

In [ ]:
df_attacker = []
df_normal = []
count = 0
for j in df_list:
#     print(j.attackerType)
    if j.attackerType.any() == 0:       
        df_temp = j.iloc[:,4:].fillna(0)
        df_temp = np.asarray(df_temp).astype('float32')
        j = np.pad(df_temp, ((0,100-j.shape[0]),(0,0)), 'constant')
#         j = torch.tensor(j, dtype=torch.long)
        df_normal.append(j.flatten())
#         print(j.flatten().shape)
    elif j.attackerType.any() == 1:
        count+=1
        df_temp = j.iloc[:,4:].fillna(0)
        df_temp = np.asarray(df_temp).astype('float32')
        j = np.pad(df_temp, ((0,100-j.shape[0]),(0,0)), 'constant')
#         j = torch.tensor(j, dtype=torch.long)
        df_attacker.append(j.flatten())

        
        
# df_normal = torch.tensor(df_normal)
# df_attacker = torch.tensor(df_attacker)
# torch.save(df_normal, 'df_normal_allscaled.pt')
# torch.save(df_attacker, 'df_attacker_allscaled.pt')

df_normal = np.asarray(df_normal)
df_attacker = np.asarray(df_attacker)

df_normal.shape
# np.save(r'C:\Users\aidan\Desktop\5G-BSM-Threat-Detection\ScenarioScaled\df_normal_allscaled_flat.npy', df_normal)
# np.save(r'C:\Users\aidan\Desktop\5G-BSM-Threat-Detection\ScenarioScaled\df_attacker_allscaled_flat.npy', df_attacker)

In [ ]:
np.save(r'C:\Users\aidan\Desktop\5G-BSM-Threat-Detection\ScenarioScaled\df_normal_allscaled_flat.npy', df_normal)
np.save(r'C:\Users\aidan\Desktop\5G-BSM-Threat-Detection\ScenarioScaled\df_attacker_allscaled_flat.npy', df_attacker)

In [ ]:
df_attacker = []
df_normal = []
count = 0
for j in df_list:
#     print(j.attackerType)
    if j.attackerType.any() == 0:       
        df_temp = j.iloc[:,4:].fillna(0)
        scaler = StandardScaler()
        df_temp[['pos0', 'pos1']] = scaler.fit_transform(df_temp[['pos0', 'pos1']])
        df_temp[['spd0', 'spd1']] = scaler.fit_transform(df_temp[['spd0', 'spd1']])
        df_temp[['pos0Close0', 'pos1Close0']] = scaler.fit_transform(df_temp[['pos0Close0', 'pos1Close0']])
        df_temp[['spd0Close0', 'spd1Close0']] = scaler.fit_transform(df_temp[['spd0Close0', 'spd1Close0']])
        df_temp[['pos0Close1', 'pos1Close1']] = scaler.fit_transform(df_temp[['pos0Close1', 'pos1Close1']])
        df_temp[['spd0Close1', 'spd1Close1']] = scaler.fit_transform(df_temp[['spd0Close1', 'spd1Close1']])
#         display(df_temp)
        
#         df_temp = np.asarray(df_temp).astype('float32')
        j = np.pad(df_temp, ((0,100-j.shape[0]),(0,0)), 'constant')
        j = torch.tensor(j, dtype=torch.long)
        df_normal.append(j)
#         print(np.asarray(j).shape)
    elif j.attackerType.any() == 1:
        count+=1
        
        df_temp = j.iloc[:,4:].fillna(0)
        scaler = StandardScaler()
        df_temp[['pos0', 'pos1']] = scaler.fit_transform(df_temp[['pos0', 'pos1']])
        df_temp[['spd0', 'spd1']] = scaler.fit_transform(df_temp[['spd0', 'spd1']])
        df_temp[['pos0Close0', 'pos1Close0']] = scaler.fit_transform(df_temp[['pos0Close0', 'pos1Close0']])
        df_temp[['spd0Close0', 'spd1Close0']] = scaler.fit_transform(df_temp[['spd0Close0', 'spd1Close0']])
        df_temp[['pos0Close1', 'pos1Close1']] = scaler.fit_transform(df_temp[['pos0Close1', 'pos1Close1']])
        df_temp[['spd0Close1', 'spd1Close1']] = scaler.fit_transform(df_temp[['spd0Close1', 'spd1Close1']])

#         df_temp = np.asarray(df_temp).astype('float32')
        j = np.pad(df_temp, ((0,100-j.shape[0]),(0,0)), 'constant')
        j = torch.tensor(j, dtype=torch.long)
        df_attacker.append(j)

        
        
# df_normal = torch.tensor(df_normal)
# df_attacker = torch.tensor(df_attacker)
# torch.save(df_normal, 'df_normal_allscaled.pt')
# torch.save(df_attacker, 'df_attacker_allscaled.pt')

df_normal = np.asarray(df_normal)
df_attacker = np.asarray(df_attacker)

df_normal.shape
np.save(r'C:\Users\aidan\Desktop\5G-BSM-Threat-Detection\ScenarioScaled\df_normal_allscaled.npy', df_normal)
np.save(r'C:\Users\aidan\Desktop\5G-BSM-Threat-Detection\ScenarioScaled\df_attacker_allscaled.npy', df_attacker)

In [ ]:
np.save(r'C:\Users\aidan\Desktop\5G-BSM-Threat-Detection\ClosestCarCSVsModel2\np_normal.npy', df_normal)
np.save(r'C:\Users\aidan\Desktop\5G-BSM-Threat-Detection\ClosestCarCSVsModel2\np_attacker.npy', df_attacker)


# ------------------------------------------Template Code-----------------------------------------

In [ ]:
import pandas as pd
from scipy.spatial import distance

# create a sample DataFrame
df = pd.DataFrame({
    'A': [1, 2, 3, 4, 4,6,7,8,9],
    'B': [2, 4, 6, 8, 7,3,7,3,8],
    'C': [3, 6, 9, 12, 3,2,9,6,4]
})

# select a row to find closest rows to
given_row = df.iloc[4]

# Find the index of the given row
row_index = df.index.get_loc(given_row.name)

# Slice the DataFrame to only consider rows above the given row
above_df = df.iloc[:row_index]

# if len(above_df) == 0:
#     return pd.DataFrame()


# select the A and B columns of the DataFrame
df_ab = above_df[['A', 'B']]

# calculate the Euclidean distance between the given row and all other rows based on A and B values
distances = distance.cdist(df_ab, [given_row[['A', 'B']]], metric='euclidean')

# get the indices of rows sorted by distance
sorted_indices = distances.argsort(axis=0).flatten()

# select the two closest rows (excluding the given row itself)
closest_indices = sorted_indices[0:3]

# get the closest rows as a DataFrame
closest_rows = df.iloc[closest_indices]

print(closest_rows)

In [ ]:
import pandas as pd
from scipy.spatial.distance import cdist

# Generate a sample DataFrame
df = pd.DataFrame({'A': [1, 1, 1, 4, 1],
                   'B': [6, 2, 2, 9, 2],
                   'C': [11, 3, 3, 14, 3]})

# Define a function to find the two closest rows above a given row based on A and B values
def find_closest_rows_above(df, row):
    # Find the index of the given row
    row_index = df.index.get_loc(row.name)
    
    # Slice the DataFrame to only consider rows above the given row
    above_df = df.iloc[:row_index]
    
    if len(above_df) == 0:
        return pd.DataFrame()
    
    # Compute the distances between the given row and all rows above it in the DataFrame based on A and B values
    distances = cdist(above_df[['A', 'B']].values, row[['A', 'B']].values.reshape(1, -1))
    
    # Sort the distances in ascending order and retrieve the index of the closest row
    closest_index = distances.argmin(axis=0)[0]
    
    # Use the index to retrieve the closest row from the DataFrame
    closest_row = above_df.iloc[closest_index]
    
    # Find the index of the second closest row
    second_closest_index = (distances != distances[closest_index]).argmax(axis=0)[0]
    
    # Use the index to retrieve the second closest row from the DataFrame
    second_closest_row = above_df.iloc[second_closest_index]
    
    return pd.concat([closest_row, second_closest_row], axis=1).T

# Define a sample row to search for
search_row = df.iloc[2] 
# For i in range(0, len(df)): 
#     search_row = df.iloc[i]
# Append the closest_rows to a new dataframe and then glue the 2 dataframes together at the end




# Find the two closest rows above the search row based on A and B values
closest_rows = find_closest_rows_above(df, search_row)

print(closest_rows)

In [ ]:
df

In [ ]:
import pandas as pd

# create a DataFrame
df = pd.DataFrame({'pos1': [25, 30, 35],
                   'pos2': [25, 30, 35],
                   'pos3': [25, 30, 35]})

# create a new DataFrame with the new columns
closest_rows = pd.DataFrame({'pos1DIFF': [50000, 60000, 70000],
                         'pos2DIFF': [1, 2, 3]})

# concatenate the new DataFrame to the existing one
result = pd.concat([df, closest_rows], axis=1)

print(result)

In [ ]:
X = []
y = []
for i in df_normal:
    X.append(np.asarray(df_normal[0].iloc[0:10,2:]))
    y.append(np.asarray(df_normal[0].iloc[10,2:]))
X = np.asarray(X)
y = np.asarray(y)
np.save(r'C:\Users\aidan\Desktop\School\5G_Networks\Final_Project\{}\X.npy'.format(start_folder), X)
np.save(r'C:\Users\aidan\Desktop\School\5G_Networks\Final_Project\{}\y.npy'.format(start_folder), y)